In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

In [2]:

def manual_data(start: str, end: str, src: str) -> pd.DataFrame:
    """Get data from """
    data_df = pd.read_csv(src, index_col=0)#, parse_dates=["Time"])

    freq='15min'
    intervals_per_hour = 4

    # temp bandaid to deal with substation data
    # data_df.index = pd.to_datetime(data_df.index, utc=True)#, format='mixed')

    # NOTE we'll adopt NEM time such that timestamps indicate the end of a 5 min interval
    # Starting with 15 min data we'll set the time index to start at the beninning of this 
    # period and then trim off the first 5 min after ffill()
    tmp = pd.date_range(start=pd.to_datetime("2021-7-1 00:00").tz_localize("UTC"), 
                        end=pd.to_datetime("2023-06-30 23:45").tz_localize("UTC"), 
                        freq=freq)
    data_df.index = tmp

    # data_df = data_df.resample("5min").ffill()#.reset_index()

    # data_df.rename(columns={data_df.columns[0]: "network_load"}, inplace=True)

    new_df = data_df.loc[start:end][1:]

    # new_df.reset_index(inplace=True)

    # temp bandaid to deal with substation data
    # new_df = new_df.rename(columns={data_df.index.name:'settlementdate'})
    # new_df = new_df.rename(columns={'index':'settlementdate'})
    return new_df, intervals_per_hour

In [34]:
zs_df, intervals_per_hour = manual_data("2022-1-1 00:00", "2022-2-1 00:00",
    src = "data/evoenergy-zone-substation-report-2021-23.csv")


for i_d, day in enumerate(range(1,3)):
    current_day_zs_df = zs_df['Belconnen'][zs_df.index.dayofyear == day]
    current_day_zs_df.rename('Current', inplace=True)
    # current_day_zs_df.index = current_day_zs_df.index.strftime('%H:%M')
    # current_day_zs_df.index = current_day_zs_df.index.time
    # time_interval = current_day_zs_df.index[1] - current_day_zs_df.index[0]
    # print(current_day_zs_df)
    # print(repr(current_day_zs_df.index))

    demand_growth = 0.05 # as a growth rate

    current_hw_MWh_per_day = 10 # will be temperature dependent
    current_hw_MW = current_hw_MWh_per_day/24
        
    curent_day_zs_sans_hw = current_day_zs_df - current_hw_MW
    curent_day_zs_sans_hw.rename('Current w/o hot water', inplace=True)

    future_zs = current_day_zs_df*(1+demand_growth)
    future_zs_sans_hw = curent_day_zs_sans_hw*(1+demand_growth)

    forecast_hw_heatpump_MWh = 40
    forecast_hw_resist_MWh = 10
    timed_hw_start = 9
    timed_hw_end = 16
    time_mask = (current_day_zs_df.index.hour >= timed_hw_start) & \
            (current_day_zs_df.index.hour < timed_hw_end)
    timed_hw_df = pd.Series(index=current_day_zs_df.index, data = 0)
    # timed_hw_df[timed_hw_df.index.hour > timed_hw_start & timed_hw_df.index.hour < timed_hw_end] == 1
    # print(timed_hw_df.between_time(timed_hw_start,timed_hw_end))
    timed_hw_df[time_mask] = 1
    forecast_hw_MW = (forecast_hw_heatpump_MWh + forecast_hw_resist_MWh) / \
                        (np.sum(timed_hw_df)/intervals_per_hour)
    timed_hw_df[time_mask] = forecast_hw_MW
    # print(timed_hw_df)

    future_zs_with_hw = future_zs_sans_hw + timed_hw_df
    future_zs_with_hw.rename('Future', inplace=True)

    # print(future_zs_with_hw)

    plotting_df = pd.concat([current_day_zs_df, curent_day_zs_sans_hw, future_zs_with_hw],axis=1)

    # # future_zs_with_hw.plot()
    # plotting_df.plot()
    # # print(plotting_df)

    future_zs_with_hw.index = current_day_zs_df.index.time

    if i_d == 0:
        many_day_df = future_zs_with_hw.rename(day, inplace=True).to_frame()
    else:
        many_day_df = many_day_df.join(future_zs_with_hw.rename(day))
print(many_day_df)


                1        2
00:15:00  19.3445  20.3105
00:30:00  19.0820  19.6805
00:45:00  18.6830  19.3760
01:00:00  18.4730  19.1345
01:15:00  18.1895  18.9455
...           ...      ...
22:45:00  24.4160  24.9935
23:00:00  23.4815  24.0695
23:15:00  22.7150  23.1035
23:30:00  21.8960  22.2005
23:45:00  21.4130  21.6545

[95 rows x 2 columns]


/tmp/ipykernel_35541/4167593100.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.142857142857143' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  timed_hw_df[time_mask] = forecast_hw_MW
/tmp/ipykernel_35541/4167593100.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.142857142857143' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  timed_hw_df[time_mask] = forecast_hw_MW
